In [1]:
# Download important libraries
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
np.random.seed(42)
tf.random.set_seed(42)
import json
import os
from os import path
import tarfile
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace, col
from functools import reduce
pipeline=None

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.1.1-bin-hadoop2.7'

In [122]:
spark.stop()

In [3]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0").getOrCreate()
spark

In [11]:
def clean_df(df):
    df = df[df.caption != '']
    
    return df.reset_index(drop=True)

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
%%timeit
rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)][:200]
df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)
    df_clean = clean_df(df)
    df_clean['place_id'] = file.split('_',1)[1]
    df_clean['place_id'] = df_clean['place_id'].str.replace('.json','')
    df_rev = pd.concat((df_rev, df_clean), ignore_index=True)

3.51 s ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
rev_dpath_jsonl = '../data/outputs/reviews/jsonl/'
os.makedirs(rev_dpath_jsonl, exist_ok=True)
rev_files_jsonl = [rev_dpath_jsonl+ f for f in os.listdir(rev_dpath_jsonl)]

rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)][:-1]
for file in rev_files:
    op_file = rev_dpath_jsonl + file.split('/reviews/')[1].split('.json')[0] + '.json'
    if op_file in rev_files_jsonl:
        continue

    with open(file) as f:
        JSON_file = json.load(f)
    with open(op_file, 'w') as outfile:
        for entry in JSON_file:
            json.dump(entry, outfile)
            outfile.write('\n')


In [117]:
df_rev

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user,census_block_group
0,ChZDSUhNMG9nS0VJQ0FnSURJME52MFhBEAE,,2 years ago,2021-03-23,4,MIGUEL BOSCAN,0,0,https://www.google.com/maps/contrib/1160597364...,10730027002
1,ChZDSUhNMG9nS0VJQ0FnSUN3a0tYS0N3EAE,Money to pawy secy y QPRyiy,2 years ago,2021-03-23,1,Prathamesh Adhikari,1,0,https://www.google.com/maps/contrib/1146220333...,10730027002
2,ChZDSUhNMG9nS0VJQ0FnSUNBOGQ2UUh3EAE,,2 years ago,2021-03-23,5,Mark Zehner,0,0,https://www.google.com/maps/contrib/1125435409...,10730027002
3,ChZDSUhNMG9nS0VJQ0FnSUN5LWZtWmZnEAE,One of my new favorite places 😋,a month ago,2021-03-23,5,Monica Crist,20,0,https://www.google.com/maps/contrib/1173007091...,10810402004
4,ChZDSUhNMG9nS0VJQ0FnSUQ4dTVxcWFREAE,I got the Peanut Butter Cup Shake and it was r...,7 months ago,2021-03-23,5,Paulina White,0,0,https://www.google.com/maps/contrib/1064251582...,10810402004
...,...,...,...,...,...,...,...,...,...,...
1388274,ChdDSUhNMG9nS0VJQ0FnSUNBaV9EMjFBRRAB,Yummy.. Nice representative. Yummy food.,4 years ago,2021-04-01,5,S Rose,0,0,https://www.google.com/maps/contrib/1147530094...,360050431007
1388275,ChdDSUhNMG9nS0VJQ0FnSUNBN09DNDZnRRAB,,4 years ago,2021-04-01,2,M Dinkins,28,0,https://www.google.com/maps/contrib/1012188078...,360050431007
1388276,ChdDSUhNMG9nS0VJQ0FnSUNROEttZjlBRRAB,I order a beef patty and coco bread and when ...,4 years ago,2021-04-01,1,nekia gager,0,0,https://www.google.com/maps/contrib/1029374057...,360050431007
1388277,ChdDSUhNMG9nS0VJQ0FnSURnNjctanhBRRAB,,5 years ago,2021-04-01,1,Clifton Smith,0,0,https://www.google.com/maps/contrib/1002325201...,360050431007


In [116]:
st_time = time.time()
rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)]
df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)

    df_rev = pd.concat((df_rev, df), ignore_index=True)
print(time.time()-st_time)

PermissionError: [Errno 13] Permission denied: '../data/outputs/reviews/jsonl'

In [115]:
st_time = time.time()
rev_dpath_jsonl = '../data/outputs/reviews/jsonl/'
rev_files_jsonl = [rev_dpath_jsonl+ f for f in os.listdir(rev_dpath_jsonl)]
dataframes = map(lambda r: spark.read.json(r), rev_files_jsonl)
union = reduce(lambda df1, df2: df1.unionAll(df2), dataframes)
df = union.collect()
print(time.time()-st_time)

2404.066113471985


In [86]:
file = rev_dpath_jsonl = '../data/outputs/reviews/jsonl/10730027002_ChIJNe0mPMkbiYgR6oApk2Pt7F8.json'

spark.read.json(file).head(2)[0]

Row(caption='', census_block_group=10730027002, id_review='ChZDSUhNMG9nS0VJQ0FnSURJME52MFhBEAE', n_photo_user=0, n_review_user=0, rating=4.0, relative_date='2 years ago', retrieval_date='2021-03-23', url_user='https://www.google.com/maps/contrib/116059736436598578008/reviews?hl=en-US', username='MIGUEL BOSCAN')

In [17]:
df

DataFrame[_corrupt_record: string]